# DATA603 Big Data Processing Project 
Group 3: Pooja Kangokar Pranesh, Yun-Zih Chen, Elizabeth Cardosa

The goal of this project is leverage big data technologies to train a model using the UCI ML Drug Review dataset to predict the star rating of drug based on the sentiment of the review. This model will then perform inference in a streaming manner on ‘real-time’ reviews coming in. This application can then be used to help potential customers understand the overall sentiment towards a drug and if it might be useful for them. 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
working_folder = "/content/drive/My Drive/UMBC Fall 2022/DATA603 Big Data Processing/Project/Data/"

# Install Libraries and Dependencies

In [3]:
# Install PySpark and Spark NLP
! pip install -qq pyspark==3.2.1 spark-nlp findspark 

     |████████████████████████████████| 281.4 MB 24 kB/s 
     |████████████████████████████████| 448 kB 61.7 MB/s 
     |████████████████████████████████| 198 kB 46.4 MB/s 


In [4]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-12-11 18:19:31--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-12-11 18:19:31--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-12-11 18:19:32--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:44

In [5]:
import pyspark.pandas as ps
import pandas as pd

In [6]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext

In [7]:
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

In [8]:
"""# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark"""

'# Import SparkSession\nfrom pyspark.sql import SparkSession\n# Create a Spark Session\nspark = SparkSession.builder.master("local[*]").getOrCreate()\n# Check Spark Session Information\nspark'

In [9]:
spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 4.2.4
Apache Spark version: 3.2.1


In [10]:
sc = SparkContext.getOrCreate();

# Read-in Dataset


## Dataset: https://archive.ics.uci.edu/ml/datasets/Drug+Review+Dataset+%28Drugs.com%29


The dataset provides patient reviews on specific drugs along with related conditions and a 10 star patient rating reflecting overall patient satisfaction. The data was obtained by crawling online pharmaceutical review sites. The intention was to study

- sentiment analysis of drug experience over multiple facets, i.e. sentiments learned on specific aspects such as effectiveness and side effects,
- the transferability of models among domains, i.e. conditions, and
- the transferability of models among different data sources (see 'Drug Review Dataset (Druglib.com)').

The data is split into a train (75%) a test (25%) partition (see publication) and stored in two .tsv (tab-separated-values) files, respectively.

Attribute Information:

1. drugName (categorical): name of drug
2. condition (categorical): name of condition
3. review (text): patient review
4. rating (numerical): 10 star patient rating
5. date (date): date of review entry
6. usefulCount (numerical): number of users who found review useful


Important notes:

When using this dataset, you agree that you
1. only use the data for research purposes
2. don't use the data for any commerical purposes
3. don't distribute the data to anyone else
4. cite us

Felix Gräßer, Surya Kallumadi, Hagen Malberg, and Sebastian Zaunseder. 2018. Aspect-Based Sentiment Analysis of Drug Reviews Applying Cross-Domain and Cross-Data Learning. In Proceedings of the 2018 International Conference on Digital Health (DH '18). ACM, New York, NY, USA, 121-125. DOI: [Web Link] 

## Load in Test Data

In [11]:
# Read in training data file
customschema = StructType([
  StructField("UniqueID", IntegerType(), True)
  ,StructField("drugName", StringType(), True)
  ,StructField("condition", StringType(), True)
  ,StructField("review", StringType(), True)
  ,StructField("rating", DoubleType(), True)
  ,StructField("date", StringType(), True)
  ,StructField("usefulCount", IntegerType(), True)
  ])

In [12]:
df_test = spark.read.format("csv")\
           .option("delimiter", "\t")\
           .option("header", "true")\
           .option("quote", "\"")\
           .option("escape", "\"")\
           .option("multiLine","true")\
           .option("quoteMode","ALL")\
           .option("mode","PERMISSIVE")\
           .option("ignoreLeadingWhiteSpace","true")\
           .option("ignoreTrailingWhiteSpace","true")\
           .option("parserLib","UNIVOCITY")\
           .schema(customschema)\
           .load(working_folder + "drugsComTest_raw.tsv")

In [13]:
df_test.count()

53766

In [14]:
df_test.show(5)

+--------+---------------+--------------------+--------------------+------+------------------+-----------+
|UniqueID|       drugName|           condition|              review|rating|              date|usefulCount|
+--------+---------------+--------------------+--------------------+------+------------------+-----------+
|  163740|    Mirtazapine|          Depression|"I&#039;ve tried ...|  10.0| February 28, 2012|         22|
|  206473|     Mesalamine|Crohn's Disease, ...|"My son has Crohn...|   8.0|      May 17, 2009|         17|
|  159672|        Bactrim|Urinary Tract Inf...|"Quick reduction ...|   9.0|September 29, 2017|          3|
|   39293|       Contrave|         Weight Loss|"Contrave combine...|   9.0|     March 5, 2017|         35|
|   97768|Cyclafem 1 / 35|       Birth Control|"I have been on t...|   9.0|  October 22, 2015|          4|
+--------+---------------+--------------------+--------------------+------+------------------+-----------+
only showing top 5 rows



## Load in and Explore Training Data

In [15]:
# Read in training data file
customschema = StructType([
  StructField("UniqueID", IntegerType(), True)
  ,StructField("drugName", StringType(), True)
  ,StructField("condition", StringType(), True)
  ,StructField("review", StringType(), True)
  ,StructField("rating", DoubleType(), True)
  ,StructField("date", StringType(), True)
  ,StructField("usefulCount", IntegerType(), True)
  ])

df = spark.read.format("csv")\
           .option("delimiter", "\t")\
           .option("header", "true")\
           .option("quote", "\"")\
           .option("escape", "\"")\
           .option("multiLine","true")\
           .option("quoteMode","ALL")\
           .option("mode","PERMISSIVE")\
           .option("ignoreLeadingWhiteSpace","true")\
           .option("ignoreTrailingWhiteSpace","true")\
           .option("parserLib","UNIVOCITY")\
           .schema(customschema)\
           .load(working_folder + "drugsComTrain_raw.tsv")

In [16]:
df.count()

161297

In [17]:
df.show(5)

+--------+--------------------+--------------------+--------------------+------+-----------------+-----------+
|UniqueID|            drugName|           condition|              review|rating|             date|usefulCount|
+--------+--------------------+--------------------+--------------------+------+-----------------+-----------+
|  206461|           Valsartan|Left Ventricular ...|"It has no side e...|   9.0|     May 20, 2012|         27|
|   95260|          Guanfacine|                ADHD|"My son is halfwa...|   8.0|   April 27, 2010|        192|
|   92703|              Lybrel|       Birth Control|"I used to take a...|   5.0|December 14, 2009|         17|
|  138000|          Ortho Evra|       Birth Control|"This is my first...|   8.0| November 3, 2015|         10|
|   35696|Buprenorphine / n...|   Opiate Dependence|"Suboxone has com...|   9.0|November 27, 2016|         37|
+--------+--------------------+--------------------+--------------------+------+-----------------+-----------+
o

### Clean Training Dataset

In [18]:
# Remove rows with null columns
df = df.dropna()
df_test = df_test.dropna()

In [19]:
df.count()

160398

In [20]:
df_test.count()

53471

In [21]:
# Drop conditions with </span> tag
df = df.where(~df.condition.contains("</span>"))

In [22]:
df_test = df_test.where(~df_test.condition.contains("</span>"))

In [23]:
df.count()

159498

In [24]:
df_test.count()

53200

In [25]:
df.groupby('rating').count().orderBy("rating", ascending=False).show()

+------+-----+
|rating|count|
+------+-----+
|  10.0|50504|
|   9.0|27219|
|   8.0|18688|
|   7.0| 9338|
|   6.0| 6254|
|   5.0| 7907|
|   4.0| 4942|
|   3.0| 6422|
|   2.0| 6833|
|   1.0|21391|
+------+-----+



In [26]:
# Average Star Rating by Condition
df.groupBy("condition").agg({'rating':'avg', 'condition':'count'}).orderBy("count(condition)",ascending=False).show()

+--------------------+----------------+------------------+
|           condition|count(condition)|       avg(rating)|
+--------------------+----------------+------------------+
|       Birth Control|           28788| 6.089933305543977|
|          Depression|            9069| 7.099459697871871|
|                Pain|            6145|7.6322213181448335|
|             Anxiety|            5904| 7.691056910569106|
|                Acne|            5588| 7.374194702934861|
|     Bipolar Disorde|            4224| 7.152698863636363|
|            Insomnia|            3673| 6.724203648243942|
|         Weight Loss|            3609| 8.051260737046274|
|             Obesity|            3568| 7.744674887892376|
|                ADHD|            3383|7.3502808158439255|
|    Diabetes, Type 2|            2554| 6.611981205951449|
|Emergency Contrac...|            2463| 8.432399512789281|
| High Blood Pressure|            2321| 6.214993537268419|
|Vaginal Yeast Inf...|            2274| 4.08223394898856

In [27]:
# Average Star Rating by Drug Name 
df.groupBy("drugName").agg({'rating':'avg', 'drugName':'count'}).orderBy("count(drugName)",ascending=False).show()

+--------------------+------------------+---------------+
|            drugName|       avg(rating)|count(drugName)|
+--------------------+------------------+---------------+
|      Levonorgestrel| 7.403470118424677|           3631|
|        Etonogestrel|  5.80517916290274|           3321|
|Ethinyl estradiol...| 5.589090909090909|           2750|
|           Nexplanon| 5.679851093531875|           2149|
|Ethinyl estradiol...|5.8362026561731435|           2033|
|Ethinyl estradiol...| 5.798783858485351|           1809|
|         Phentermine|  8.78218465539662|           1538|
|          Sertraline| 7.492239467849224|           1353|
|        Escitalopram| 7.846930846930847|           1287|
|              Mirena| 6.595570139458572|           1219|
|            Implanon| 6.073059360730594|           1095|
|          Gabapentin| 7.436660268714012|           1042|
|           Bupropion| 7.442477876106195|           1017|
|         Venlafaxine|6.7895256916996045|           1012|
|          Mic

In [28]:
pd_df_train = df.toPandas()

In [29]:
pd_df_test = df_test.toPandas()

# Use TextBlob to Extract Sentiments
John Snow Labs sentiment models do not provide us with a continuous sentiment score, but simply postive or negative labels. Also, due to lazy evaluation in Spark the inference transformations blow up the size of the dataframe causing it to be unworkable on our UMBC Colab instances. Because of this, we have opted to use the TextBlob library to obtain the sentiment polarity and Swifter to parallelize the inference operation to train our final model. 

In [30]:
!pip install swifter -qq

     |████████████████████████████████| 830 kB 5.0 MB/s 
     |████████████████████████████████| 280 kB 52.5 MB/s 
     |████████████████████████████████| 1.6 MB 51.3 MB/s 


In [31]:
from textblob import TextBlob
import swifter

In [32]:
def get_sentiment(text):
  return TextBlob(text).sentiment.polarity

In [33]:
pd_df_train['sentiment'] = pd_df_train['review'].swifter.apply(get_sentiment)

Pandas Apply:   0%|          | 0/159498 [00:00<?, ?it/s]

In [34]:
pd_df_train.head()

,UniqueID,drugName,condition,review,rating,date,usefulCount,sentiment
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,0.000000
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,0.168333
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,0.067210
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,0.179545
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,0.194444


In [35]:
pd_df_train.sentiment.describe()

count    159498.000000
mean          0.064715
std           0.222336
min          -1.000000
25%          -0.043333
50%           0.057975
75%           0.173611
max           1.000000
Name: sentiment, dtype: float64

In [36]:
pd_df_test['sentiment'] = pd_df_test['review'].swifter.apply(get_sentiment)

Pandas Apply:   0%|          | 0/53200 [00:00<?, ?it/s]

In [37]:
pd_df_test.head()

,UniqueID,drugName,condition,review,rating,date,usefulCount,sentiment
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22,0.000000
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17,0.566667
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3,0.333333
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35,0.139063
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4,0.260926


In [38]:
import csv

In [39]:
pd_df_train.to_csv(working_folder + "drug_reviews_with_sentiment_train.csv", index=False, sep='|', quoting=csv.QUOTE_MINIMAL)
pd_df_test.to_csv(working_folder + "drug_reviews_with_sentiment_test.csv", index=False, sep='|', quoting=csv.QUOTE_MINIMAL)

# Testing John Snow Labs pretrained sentiment models pipeline


https://nlp.johnsnowlabs.com/

Medium Article: 
https://medium.com/analytics-vidhya/sentiment-analysis-with-sparknlp-couldnt-be-easier-2a8ea3b728a0

John Snow Labs Reference Notebook: 
https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/quick_start_google_colab.ipynb#scrollTo=tyMMD_upEfIa

This model using BioBERT would potentially perform better, but it is not free-tier:
https://nlp.johnsnowlabs.com/2022/07/28/bert_sequence_classifier_drug_reviews_webmd_en_3_0.html


### Use Twitter Sentiment Analysis Model: analyze_sentimentdl_use_twitter 

Model: https://nlp.johnsnowlabs.com/2021/01/18/sentimentdl_use_twitter_en.html

Universal Sentence Encoder: https://nlp.johnsnowlabs.com/2020/04/17/tfhub_use.html

In [40]:
"""pipeline = PretrainedPipeline('analyze_sentimentdl_use_twitter', 'en')
pipeline.model.stages
# rename the text column as 'text', pipeline expects 'text' 
df_result = pipeline.transform(df.withColumnRenamed("review", "text"))
# Extract results from the "sentiments" column
df_twitter_sentiments = df_result.withColumn("sentiment", explode('sentiment.result')).drop(*['document','sentence_embeddings'])"""

'pipeline = PretrainedPipeline(\'analyze_sentimentdl_use_twitter\', \'en\')\npipeline.model.stages\n# rename the text column as \'text\', pipeline expects \'text\' \ndf_result = pipeline.transform(df.withColumnRenamed("review", "text"))\n# Extract results from the "sentiments" column\ndf_twitter_sentiments = df_result.withColumn("sentiment", explode(\'sentiment.result\')).drop(*[\'document\',\'sentence_embeddings\'])'


A vast majority of the reviews are negative


+---------+------+<br>
|sentiment| count|<br>
+---------+------+<br>
| positive| 31299|<br>
|  neutral|  6568|<br>
| negative|123430|<br>
+---------+------+



In [41]:
# took 20 minutes to run
#df_twitter_sentiments.groupBy('sentiment').count().show()

### Use RoBERTa Sentiment Classifier: roberta_classifier_autotrain_sentiment_polarity_918130222

Model: https://nlp.johnsnowlabs.com/2022/09/19/roberta_classifier_autotrain_sentiment_polarity_918130222_en.html

HuggingFace: https://huggingface.co/docs/transformers/model_doc/roberta

Breakdown how pretrained pipeline works under the hood: https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/SENTIMENT_EN.ipynb

In [42]:
documentAssembler = DocumentAssembler() \
        .setInputCol("review") \
        .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols("document") \
    .setOutputCol("token")

seq_classifier = RoBertaForSequenceClassification.pretrained("roberta_classifier_autotrain_sentiment_polarity_918130222","en") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("sentiment")

roberta_classifier_autotrain_sentiment_polarity_918130222 download started this may take some time.
Approximate size to download 294.8 MB
[OK!]


In [43]:
nlp_pipeline = Pipeline(stages=[documentAssembler, tokenizer, seq_classifier])

In [44]:
df_train = nlp_pipeline.fit(df).transform(df)

In [45]:
df_train = df_train.withColumn("sentiment", explode('sentiment.result')).drop('document','token','class')

In [46]:
df_train.show()

+--------+--------------------+--------------------+--------------------+------+------------------+-----------+---------+
|UniqueID|            drugName|           condition|              review|rating|              date|usefulCount|sentiment|
+--------+--------------------+--------------------+--------------------+------+------------------+-----------+---------+
|  206461|           Valsartan|Left Ventricular ...|"It has no side e...|   9.0|      May 20, 2012|         27|      1.0|
|   95260|          Guanfacine|                ADHD|"My son is halfwa...|   8.0|    April 27, 2010|        192|      1.0|
|   92703|              Lybrel|       Birth Control|"I used to take a...|   5.0| December 14, 2009|         17|      0.0|
|  138000|          Ortho Evra|       Birth Control|"This is my first...|   8.0|  November 3, 2015|         10|      1.0|
|   35696|Buprenorphine / n...|   Opiate Dependence|"Suboxone has com...|   9.0| November 27, 2016|         37|      1.0|
|  155963|              

In [47]:
df_train = df_train.withColumn("sentiment", df_train["sentiment"].cast(DoubleType()))

In [48]:
df_train.count()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
import py4j.protocol  
from py4j.protocol import Py4JJavaError  
from py4j.java_gateway import JavaObject  
from py4j.java_collections import JavaArray, JavaList

from pyspark import RDD, SparkContext  
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer


# Helper function to convert python object to Java objects
def _to_java_object_rdd(rdd):  
    """ Return a JavaRDD of Object by unpickling
    It will convert each Python object into Java object by Pyrolite, whenever the
    RDD is serialized in batch or not.
    """
    rdd = rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
    return rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)

# First you have to convert it to an RDD 
JavaObj = _to_java_object_rdd(small_df.rdd)

# Now we can run the estimator
sc._jvm.org.apache.spark.util.SizeEstimator.estimate(JavaObj)

In [ ]:
# First you have to convert it to an RDD 
JavaObj = _to_java_object_rdd(df_train.rdd)

# Now we can run the estimator
sc._jvm.org.apache.spark.util.SizeEstimator.estimate(JavaObj)

In [ ]:
#df_train.write.mode('overwrite').parquet(working_folder + "drug_reviews_with_sentiment_train.parquet")

In [ ]:
# Drop rows with missing values
df_test = df_test.dropna()

In [ ]:
## Drop rows where condition contains irrelevant strings
df_test = df_test.where(~df_test.condition.contains("</span>"))

In [ ]:
df_test.count()

In [ ]:
df_test = nlp_pipeline.fit(df_test).transform(df_test)

In [ ]:
df_test = df_test.withColumn("sentiment", explode('class.result')).drop(*['token','class','document'])

In [ ]:
df_test.show()

In [ ]:
# Write complete dataframe to disk
#df_test.write.csv(working_folder + "drug_reviews_with_sentiment_test.csv")
# Write complete dataframe to disk
#pd_df_test = df_test.toPandas()
#pd_df_test.to_csv(working_folder + "drug_reviews_with_sentiment_test.csv")
df_test.write.mode('overwrite').parquet(working_folder + "drug_reviews_with_sentiment_test.parquet")